In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [2]:
#best = pd.read_csv('./v4_v10_01_rapids.csv')

In [3]:
best = pd.read_csv('./submission_rapids_ensemble.csv')

In [4]:
npzs = glob.glob('*.npz')

In [5]:
len(npzs)

7

In [6]:
npzs

['fold_3_feat_0_epoch_23_batch_81_loss_0.1402377337217331.npz',
 'fold_2_feat_0_epoch_23_batch_81_loss_0.13422970473766327.npz',
 'fold_4_feat_0_epoch_34_batch_81_loss_0.14502757787704468.npz',
 'fold_1_feat_0_epoch_23_batch_81_loss_0.1413227617740631.npz',
 'fold_0_feat_0_epoch_15_batch_81_loss_0.1450279951095581.npz',
 'fold_3_feat_0_epoch_13_batch_81_loss_0.13834697008132935.npz',
 'fold_2_feat_0_epoch_32_batch_81_loss_0.1342601776123047.npz']

In [7]:
for idx, npz in enumerate(npzs):
    preds = np.load(npz)
    if idx == 0:
        y_preds = preds['y_pred']
    else:
        y_preds += preds['y_pred']
#@ids = preds['ids']

In [8]:
y_preds /= 7

In [9]:
mask = best['Id'].apply(lambda x: True if 'age' in x else False)

In [10]:
best.loc[mask, 'Predicted']

0        56.298264
5        64.031325
10       38.098119
15       51.114563
20       51.614733
           ...    
29360    38.780901
29365    53.905071
29370    41.037846
29375    39.141255
29380    44.017790
Name: Predicted, Length: 5877, dtype: float64

In [11]:
#best.loc[mask, 'Predicted'] = 0.5 * best.loc[mask, 'Predicted'] + 0.5 * y_preds[:, 0]
best.loc[mask, 'Predicted'] = y_preds[:, 0]

In [12]:
best.to_csv('./rapids_v38_kd_norm_7_best_replaced_know_distill_0_10.csv', index=False)

In [13]:
foo = pd.read_csv('./best.csv')

In [14]:
np.corrcoef(best['Predicted'], foo['Predicted'])

array([[1.        , 0.99959968],
       [0.99959968, 1.        ]])

In [18]:
sample_sub

,Id,Predicted
0,10003_age,55.708374
1,10003_domain1_var1,53.683960
2,10003_domain1_var2,60.396282
3,10003_domain2_var1,49.479599
4,10003_domain2_var2,53.225880
...,...,...
29380,21753_age,35.301876
29381,21753_domain1_var1,47.904369
29382,21753_domain1_var2,61.657104
29383,21753_domain2_var1,43.890442


In [11]:
ensemble = sample_sub.copy()

In [12]:
ensemble['Predicted'] = 0.4 * sample_sub['Predicted'] + 0.6 * new_sub['Predicted']

In [13]:
ensemble.to_csv('./v4_fold_all_rapids.csv', index=False)

In [19]:
import cudf